# AIA Machine Learning

### Sepsis Prediction Overview

Sepsis is the leading cause of mortality in the United States and the most expensive condition associated with in-hospital stays, accounting for 6.2% (nearly $24 billion) of total hospital costs. In particular, Septic shock, the most advanced complication of sepsis due to severe abnormalities of circulation and/or cellular metabolism, reaches a mortality rate as high as 50% and the annual incidence keeps rising. It is estimated that as many as 80% of sepsis deaths could be prevented with early diagnosis and intervention. Indeed, prior studies have demonstrated that early diagnosis and treatment of septic shock can significantly decrease patients’ mortality and shorten their length of stay. In this capstone, the task is to build a machine learning model for accurate early diagnosis of septic shock.

*This Jupyter Notebook uses existing python files for the below RTP mining activities:*
https://github.com/farzanehkh/RTP-mining-ICHI18

In [3]:
import RTP
import numpy as np
import pandas as pd

from tensorflow import keras
import tensorflow.random as tf_random

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import recall_score, accuracy_score
from sklearn.metrics import precision_score, f1_score

tf_random.set_seed(2022)
random_state = 2022

In [2]:
# Get data.
shock = pd.read_csv('mimic/MIMIC_III_shock.csv')
nonshock = pd.read_csv('mimic/MIMIC_III_nonshock.csv')

# Get important features only.
positive = shock.loc[:,['VisitIdentifier', 
                        'MinutesFromArrival', 
                        'SystolicBP', 
                        'HeartRate', 
                        'RespiratoryRate', 
                        'Temperature', 
                        'WBC', 
                        'ShockTime']]
negative = nonshock.loc[:,['VisitIdentifier', 
                           'MinutesFromArrival', 
                           'SystolicBP', 
                           'HeartRate', 
                           'RespiratoryRate', 
                           'Temperature', 
                           'WBC', 
                           'LastMinute']]

# Change feature data types to match between datasets.
negative[['LastMinute']] = negative[['LastMinute']].astype('int64')

# Change feature names to match between datasets.
positive.rename(columns = {'ShockTime': 'DiagnosisMinutes'}, inplace = True)
negative.rename(columns = {'LastMinute': 'DiagnosisMinutes'}, inplace = True)

# Retain only those rows that occurred at least 24 hours (1440 minute) before diagnosis/end-visit time.
p_len = len(positive)
n_len = len(negative)
positive = positive[positive['DiagnosisMinutes'] - positive['MinutesFromArrival'] >= 1440]
negative = negative[negative['DiagnosisMinutes'] - negative['MinutesFromArrival'] >= 1440]

# Drop time features.
positive = positive.drop(['DiagnosisMinutes', 'MinutesFromArrival'], axis='columns')
negative = negative.drop(['DiagnosisMinutes', 'MinutesFromArrival'], axis='columns')

# Add label 'Diagnosis', (positive: 1, negative: 0).
positive['Diagnosis'] = 1
negative['Diagnosis'] = 0

# Combine the positive and negative data into a single dataframe.
visits = pd.concat([positive, negative])

In [3]:
# Seperate the features and the labels into their respective X and y lists.
X, y = [], []
for visit in visits['VisitIdentifier'].unique():
    df_visit = visits[visits['VisitIdentifier'] == visit]
    y.append(int(df_visit.tail(1)['Diagnosis']))
    X.append(df_visit[['SystolicBP', 'HeartRate', 'RespiratoryRate', 'Temperature', 'WBC']].values.tolist())

# Pad the nested lists of feature vectors in X to match the length of the longest list of feature vectors in X.
X_padded = keras.preprocessing.sequence.pad_sequences(X, dtype='float64', padding="post", value=-10.0)

# Convert lists to numpy ndimensional arrays.
X_padded = np.asarray(X_padded)
y = np.asarray(y) 

# Split X and y data sets into train and test data sets.
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.20, random_state=random_state, shuffle=True)

In [4]:
# Implement a sequential neural network, 
# add a masking layer for padded values, 
# add an LSTM recurrent neural network layer, 
# and finally add a dense layer with one output neuron.
model = keras.Sequential()
model.add(keras.layers.Masking(mask_value=-10.0, input_shape=X_train.shape[-2:]))
model.add(keras.layers.LSTM(200))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compile the sequential model.
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss='binary_crossentropy')

# Trains the model and returns a dataframe of metrics scores
def get_metrics(batch, df):
    # Fit data to the model, generate predictions, and simplify array shape. 
    model.fit(X_train, y_train, epochs=10, batch_size=batch, validation_data=[X_test, y_test])
    y_preds = model.predict(X_test).ravel() # Flattens a multidimensional array to one dimension.
    preds = np.squeeze(y_preds) # Removes single-dimensional entries from the shape of an array.
    
    # Convert prediction results into binary values.
    preds = np.where(preds > 0.5, 1, 0)
    
    # Generate metric scores.
    auc_score=roc_auc_score(y_test, y_preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f_measure = f1_score(y_test, preds)
    
    # Populate the dataframe with the generated metric scores.
    df.loc[len(df)] = ['LSTM']+[batch]+[accuracy]+[precision]+[recall]+[f_measure]+[auc_score]
    
    return df

In [5]:
# Create a dataframe to store the returned metric scores.
# Train the model using batch size 64 and save the metric scores to a csv file.
lstm_metrics = pd.DataFrame(columns = ['Method', 'batch', 'accuracy', 'precision','recall','f1 score', 'auc roc'])
lstm_metrics = get_metrics(64, lstm_metrics)
lstm_metrics.to_csv('metrics/sepsis_lstm_metrics.csv', index=False)

Epoch 1/10
10/10 [==============================] - 316s 31s/step - loss: 0.7476 - val_loss: 0.6967
Epoch 2/10
10/10 [==============================] - 307s 31s/step - loss: 0.6885 - val_loss: 0.7015
Epoch 3/10
10/10 [==============================] - 307s 31s/step - loss: 0.6869 - val_loss: 0.6867
Epoch 4/10
10/10 [==============================] - 303s 31s/step - loss: 0.6844 - val_loss: 0.6826
Epoch 5/10
10/10 [==============================] - 303s 30s/step - loss: 0.6715 - val_loss: 0.6811
Epoch 6/10
10/10 [==============================] - 299s 30s/step - loss: 0.6706 - val_loss: 0.6945
Epoch 7/10
10/10 [==============================] - 297s 30s/step - loss: 0.6776 - val_loss: 0.6926
Epoch 8/10
10/10 [==============================] - 299s 30s/step - loss: 0.6718 - val_loss: 0.6682
Epoch 9/10
10/10 [==============================] - 298s 30s/step - loss: 0.6594 - val_loss: 0.6884
Epoch 10/10
5/5 [==============================] - 32s 6s/step


In [6]:
# Retrieve the previous metric scores from the saved csv file.
# Train the model using batch size 128 and save the accumulated metric scores to a csv file.
lstm_metrics = pd.read_csv('metrics/sepsis_lstm_metrics.csv')
lstm_metrics = get_metrics(128, lstm_metrics)
lstm_metrics.to_csv('metrics/sepsis_lstm_metrics.csv', index=False)

Epoch 1/10
5/5 [==============================] - 291s 60s/step - loss: 0.6394 - val_loss: 0.6508
Epoch 2/10
5/5 [==============================] - 299s 59s/step - loss: 0.6284 - val_loss: 0.6645
Epoch 3/10
5/5 [==============================] - 309s 60s/step - loss: 0.6259 - val_loss: 0.6536
Epoch 4/10
5/5 [==============================] - 297s 59s/step - loss: 0.6297 - val_loss: 0.6342
Epoch 5/10
5/5 [==============================] - 300s 59s/step - loss: 0.6010 - val_loss: 0.6327
Epoch 6/10
5/5 [==============================] - 301s 63s/step - loss: 0.5933 - val_loss: 0.6172
Epoch 7/10
5/5 [==============================] - 304s 59s/step - loss: 0.5805 - val_loss: 0.6241
Epoch 8/10
5/5 [==============================] - 283s 58s/step - loss: 0.5619 - val_loss: 0.6116
Epoch 9/10
5/5 [==============================] - 297s 61s/step - loss: 0.5707 - val_loss: 0.6070
Epoch 10/10
5/5 [==============================] - 33s 7s/step


In [6]:
# Retrieve the previous metric scores from the saved csv file.
# Train the model using batch size 256 and save the accumulated metric scores to a csv file.
lstm_metrics = pd.read_csv('metrics/sepsis_lstm_metrics.csv')
lstm_metrics = get_metrics(256, lstm_metrics)
lstm_metrics.to_csv('metrics/sepsis_lstm_metrics.csv', index=False)

Epoch 1/10
3/3 [==============================] - 1671s 456s/step - loss: 1.1010 - val_loss: 0.7332
Epoch 2/10
3/3 [==============================] - 2031s 901s/step - loss: 0.7139 - val_loss: 0.7175
Epoch 3/10
3/3 [==============================] - 1015s 414s/step - loss: 0.7271 - val_loss: 0.6943
Epoch 4/10
3/3 [==============================] - 1806s 661s/step - loss: 0.6940 - val_loss: 0.6886
Epoch 5/10
3/3 [==============================] - 1395s 598s/step - loss: 0.6858 - val_loss: 0.6884
Epoch 6/10
3/3 [==============================] - 1965s 880s/step - loss: 0.6814 - val_loss: 0.6854
Epoch 7/10
3/3 [==============================] - 1782s 703s/step - loss: 0.6831 - val_loss: 0.6702
Epoch 8/10
3/3 [==============================] - 1258s 340s/step - loss: 0.6703 - val_loss: 0.6720
Epoch 9/10
3/3 [==============================] - 2452s 708s/step - loss: 0.6692 - val_loss: 0.6644
Epoch 10/10
5/5 [==============================] - 39s 7s/step


In [5]:
# Create a dataframe to store classifier, trial parameters and resulting metric scores.
rtp_metrics = pd.DataFrame(columns = ['Method', 
                              'model', 
                              'max gap', 
                              'sup pos', 
                              'sup neg', 
                              'accuracy', 
                              'precision',
                              'recall',
                              'f1 score', 
                              'auc roc'])

# Save dataframe to csv file.
rtp_metrics.to_csv('metrics/sepsis_rtp_metrics.csv', index=False)

# Runs RTP mining program and returns results.
def get_rtp_metrics(clf, df):
    for gap in (4,5,6,7,8,9,10):
        for pos_sup in (0.1,0.15,0.2,0.25,0.3):
            for neg_sup in (0.1,0.15,0.2,0.25,0.3):
                df.loc[len(df)] = ['RTP']+[clf]+[gap]+[pos_sup]+[neg_sup]+ RTP.main(gap, pos_sup, neg_sup, clf)
    return df

In [ ]:
# Retrieve dataframe from csv file.
# Run RTP mining program fitted with support vector machine classifier.
# Save results back to the csv file.
rtp_metrics = pd.read_csv('metrics/sepsis_rtp_metrics.csv')
rtp_metrics = get_rtp_metrics('svm', rtp_metrics)
rtp_metrics.to_csv('metrics/sepsis_rtp_metrics.csv', index=False)

In [ ]:
# Retrieve dataframe from csv file.
# Run RTP mining program fitted with logistic regression classifier.
# Save results back to the csv file.
rtp_metrics = pd.read_csv('metrics/sepsis_rtp_metrics.csv')
rtp_metrics = get_rtp_metrics('lr', rtp_metrics)
rtp_metrics.to_csv('metrics/sepsis_rtp_metrics.csv', index=False)

In [9]:
# Display RTP results

rtp_metrics = pd.read_csv('metrics/sepsis_rtp_metrics.csv')
rtp_metrics.round(3)

,Method,model,max gap,sup pos,sup neg,accuracy,precision,recall,f1 score,auc roc
0,RTP,svm,4,0.1,0.10,0.841,0.768,0.975,0.859,0.882
1,RTP,svm,4,0.1,0.15,0.841,0.772,0.967,0.858,0.887
2,RTP,svm,4,0.1,0.20,0.839,0.770,0.967,0.857,0.885
3,RTP,svm,4,0.1,0.25,0.839,0.770,0.967,0.857,0.885
4,RTP,svm,4,0.1,0.30,0.839,0.770,0.967,0.857,0.885
...,...,...,...,...,...,...,...,...,...,...
345,RTP,lr,10,0.3,0.10,0.828,0.823,0.835,0.829,0.911
346,RTP,lr,10,0.3,0.15,0.829,0.827,0.833,0.830,0.912
347,RTP,lr,10,0.3,0.20,0.835,0.829,0.846,0.837,0.912
348,RTP,lr,10,0.3,0.25,0.835,0.829,0.846,0.837,0.912


In [10]:
# Display LSTM results

lstm_metrics = pd.read_csv('metrics/sepsis_lstm_metrics.csv')
lstm_metrics.round(3)

,Method,batch,accuracy,precision,recall,f1 score,auc roc
0,LSTM,64,0.644,0.620,0.807,0.702,0.681
1,LSTM,128,0.775,0.813,0.735,0.772,0.813
2,LSTM,256,0.612,0.733,0.398,0.516,0.704


In [11]:
# Display the RTP rows with the highest metric scores.

for metric in ('accuracy', 'precision', 'recall', 'f1 score', 'auc roc'):
    print(f"Highest RTP {metric} score:")
    display(rtp_metrics[rtp_metrics[metric] == rtp_metrics[metric].max()])
    print()

Highest RTP accuracy score:


,Method,model,max gap,sup pos,sup neg,accuracy,precision,recall,f1 score,auc roc
25,RTP,svm,5,0.1,0.1,0.841772,0.77,0.974684,0.860335,0.883355



Highest RTP precision score:


,Method,model,max gap,sup pos,sup neg,accuracy,precision,recall,f1 score,auc roc
222,RTP,lr,5,0.3,0.20,0.837975,0.831266,0.848101,0.839599,0.916055
223,RTP,lr,5,0.3,0.25,0.837975,0.831266,0.848101,0.839599,0.916879
224,RTP,lr,5,0.3,0.30,0.837975,0.831266,0.848101,0.839599,0.916879
247,RTP,lr,6,0.3,0.20,0.837975,0.831266,0.848101,0.839599,0.915687
248,RTP,lr,6,0.3,0.25,0.837975,0.831266,0.848101,0.839599,0.915799
249,RTP,lr,6,0.3,0.30,0.837975,0.831266,0.848101,0.839599,0.915799
272,RTP,lr,7,0.3,0.20,0.837975,0.831266,0.848101,0.839599,0.915687
273,RTP,lr,7,0.3,0.25,0.837975,0.831266,0.848101,0.839599,0.915799
274,RTP,lr,7,0.3,0.30,0.837975,0.831266,0.848101,0.839599,0.915799
297,RTP,lr,8,0.3,0.20,0.837975,0.831266,0.848101,0.839599,0.915687



Highest RTP recall score:


,Method,model,max gap,sup pos,sup neg,accuracy,precision,recall,f1 score,auc roc
50,RTP,svm,6,0.1,0.1,0.840506,0.767396,0.977215,0.859688,0.881923
75,RTP,svm,7,0.1,0.1,0.840506,0.767396,0.977215,0.859688,0.882641
100,RTP,svm,8,0.1,0.1,0.840506,0.767396,0.977215,0.859688,0.882641



Highest RTP f1 score score:


,Method,model,max gap,sup pos,sup neg,accuracy,precision,recall,f1 score,auc roc
25,RTP,svm,5,0.1,0.1,0.841772,0.77,0.974684,0.860335,0.883355



Highest RTP auc roc score:


,Method,model,max gap,sup pos,sup neg,accuracy,precision,recall,f1 score,auc roc
208,RTP,lr,5,0.15,0.25,0.836709,0.827586,0.850633,0.838951,0.916936
209,RTP,lr,5,0.15,0.30,0.836709,0.827586,0.850633,0.838951,0.916936


In [12]:
# Display the LSTM rows with the highest metric scores.

for metric in ('accuracy', 'precision', 'recall', 'f1 score', 'auc roc'):
    print(f"Highest LSTM {metric} score:")
    display(lstm_metrics[lstm_metrics[metric] == lstm_metrics[metric].max()])
    print()

Highest LSTM accuracy score:


,Method,batch,accuracy,precision,recall,f1 score,auc roc
1,LSTM,128,0.775,0.813333,0.73494,0.772152,0.812549



Highest LSTM precision score:


,Method,batch,accuracy,precision,recall,f1 score,auc roc
1,LSTM,128,0.775,0.813333,0.73494,0.772152,0.812549



Highest LSTM recall score:


,Method,batch,accuracy,precision,recall,f1 score,auc roc
0,LSTM,64,0.64375,0.62037,0.807229,0.701571,0.681271



Highest LSTM f1 score score:


,Method,batch,accuracy,precision,recall,f1 score,auc roc
1,LSTM,128,0.775,0.813333,0.73494,0.772152,0.812549



Highest LSTM auc roc score:


,Method,batch,accuracy,precision,recall,f1 score,auc roc
1,LSTM,128,0.775,0.813333,0.73494,0.772152,0.812549


In [13]:
# Determine the best settings for each method and model.

from collections import defaultdict

best_batch = []
print('LSTM settings')
print('-'*30)
for metric in ('accuracy', 'precision', 'recall', 'f1 score', 'auc roc'):
    print(f"\tBest {metric} settings")
    best_param = lstm_metrics[lstm_metrics[metric] == lstm_metrics[metric].max()]['batch'].mode()
    best_batch.append(best_param[0])
    print(f"\t{'batch'}: {best_param[0]}")
    print()
    
print(f"\t\tBest overall settings")
print(f"\t\tbatch: {max(best_batch, key = best_batch.count)}")
print(f"\n")

for model in ('svm', 'lr'):
    best_params = defaultdict(list)
    print(f"RTP-{model} settings")
    print('-'*30)
    
    rtp_model = rtp_metrics[rtp_metrics['model'] == model]
    for metric in ('accuracy', 'precision', 'recall', 'f1 score', 'auc roc'):
        print(f"\tBest {metric} settings")
        for param in ('max gap', 'sup pos', 'sup neg'):
            best_param = rtp_model[rtp_model[metric] == rtp_model[metric].max()][param].mode()
            best_params[param].append(best_param[0])
            print(f"\t{param}: {best_param[0]}")
        print()
    print(f"\t\tBest overall settings")
    for k, v in best_params.items():
        print(f"\t\t{k}: {max(v, key = v.count)}")
    print(f"\n")

LSTM settings
------------------------------
	Best accuracy settings
	batch: 128

	Best precision settings
	batch: 128

	Best recall settings
	batch: 64

	Best f1 score settings
	batch: 128

	Best auc roc settings
	batch: 128

		Best overall settings
		batch: 128


RTP-svm settings
------------------------------
	Best accuracy settings
	max gap: 5
	sup pos: 0.1
	sup neg: 0.1

	Best precision settings
	max gap: 5
	sup pos: 0.1
	sup neg: 0.15

	Best recall settings
	max gap: 6
	sup pos: 0.1
	sup neg: 0.1

	Best f1 score settings
	max gap: 5
	sup pos: 0.1
	sup neg: 0.1

	Best auc roc settings
	max gap: 4
	sup pos: 0.15
	sup neg: 0.25

		Best overall settings
		max gap: 5
		sup pos: 0.1
		sup neg: 0.1


RTP-lr settings
------------------------------
	Best accuracy settings
	max gap: 5
	sup pos: 0.3
	sup neg: 0.2

	Best precision settings
	max gap: 5
	sup pos: 0.3
	sup neg: 0.2

	Best recall settings
	max gap: 4
	sup pos: 0.15
	sup neg: 0.15

	Best f1 score settings
	max gap: 5
	sup pos: 0.

In [14]:
# Determine the methods with the highest metric scores.

highest_metrics = pd.DataFrame(columns = ['Method', 'Metric', 'Score'])

for metric in ['accuracy', 'precision','recall', 'f1 score', 'auc roc']:
    rtp_max = rtp_metrics[metric].max()
    lstm_max = lstm_metrics[metric].max()
    
    if rtp_max >= lstm_max:
        model = rtp_metrics[rtp_metrics[metric] == rtp_metrics[metric].max()]['model'].iloc[0]
        highest_metrics.loc[len(highest_metrics)] = [f"RTP-{model}"]+[metric]+[rtp_max]
    else:
        batch = lstm_metrics[lstm_metrics[metric] == lstm_metrics[metric].max()]['batch'].iloc[0]
        highest_metrics.loc[len(highest_metrics)] = [f"LSTM-{batch}"]+[metric]+[lstm_max]

In [39]:
# Display which methods had the highest metric scores and give evaluative summary below.

from IPython.display import Markdown, display

display(Markdown("## Results and Final Report"))
print()
display(Markdown("#### Machine Learning Methods Compared\n\
* Recent Temporal Pattern (RTP)\n\
  * Support Vector Machine (SVM)\n\
  * Logistic Regression (LR)\n\
* Long Short-Term Memory (LSTM)"))
print()
display(Markdown('#### Methods with the highest metric scores'))
display(highest_metrics.round(3))

## Results and Final Report

#### Machine Learning Methods Compared
* Recent Temporal Pattern (RTP)
  * Support Vector Machine (SVM)
  * Logistic Regression (LR)
* Long Short-Term Memory (LSTM)

#### Methods with the highest metric scores

,Method,Metric,Score
0,RTP-svm,accuracy,0.842
1,RTP-lr,precision,0.831
2,RTP-svm,recall,0.977
3,RTP-svm,f1 score,0.860
4,RTP-lr,auc roc,0.917


#### Best Overall Settings and Parameters
* RTP-SVM
  * kernel: poly
  * max gap: 5
  * sup pos: 0.1
  * sup neg: 0.1
* RTP-LR
  * max gap: 5
  * sup pos: 0.3
  * sup neg: 0.2
* LSTM
  * batch: 128

#### Metrics Evaluation
* True Positive Rate (TP) - *The probability that an actual positive correctly tests positive.*
* True Negative Rate (TN) - *The probability that an actual negative correctly tests negative.*
* False Positive Rate (FP) - *The probability that an actual negative wrongly tests positive.*
* False Negative Rate (FN) - *The probability that an actual positive wrongly tests negative.*

##### Accuracy: (TP + TN) / (TP + FP + FN + TN) 
* RTP-SVM correctly predicted a combined 84% of positive and negative cases. Not very informative for sepsis diagnosis due to the fact that false negatives (sepsis patients misidentified as non-sepsis patients) have a much higher cost than false positives (non-sepsis patients misidentified as sepsis patients) and Accuracy doesn't distinguish between the two.

##### Precision: TP / (TP + FP) 
* RTP-LR correctly classified 83% of its positive predictions. This tells us that our sepsis positive predictions are reasonably reliable as long as the counter-measures for sepsis patients are not particularly invasive or resource intensive since misidentified non-sepsis patients would receive the same treatment as correctly identified sepsis patients.

##### Recall: TP / (TP + FN) 
* RTP-SVM correctly classified 97% of all positive cases. This is the most important metric for sepsis diagnosis since the accurate diagnosis of sepsis positive patients is the priority here. As such with a 97% Recall rate we are reasonably certain of our ability to intercede on behalf of sepsis patients when necessary.

##### F1-score: 2 x (Recall x Precision) / (Recall + Precision) 
* RTP-SVM has a fairly high 0.86 weighted average score between Recall and Precision. This is the ability to both identify sepsis cases (Recall) and be accurate with the cases it predicts as sepsis (Precision). In other words, the confidence we have in our ability to intervene on behalf of and spend resources on those patients and only those patients who truly need intervention.

##### AUC-ROC: Sensitivity (TP) vs (1 − Specificity (TN))
* RTP-LR has the highest AUC-ROC score of 0.91 indicating its superior performance over LSTM at distinguishing between patients with sepsis and patients without sepsis at various thresholds.

#### Summary:

Having completed a rigorous exploration of the sepsis EHR data using the methods RTP-SVM, RTP-LR, and LSTM, we have found that RTP out-performed LSTM on all of the relevant metrics. Further, the fitted models, SVM and LR, used in conjunction with RTP, each showed promise by achieving high scores in the various metrics. Of these, it is our conclusion that RTP-SVM is the most promising method due to the fact that it claims the high scores for the most relevant metrics for sepsis prediction, the Recall and F1-score. Using these scores, we can be moderately confident in our ability to intervene on behalf of sepsis patients while keeping unnecessary intervention levels to a reasonable minimum.

#### Proposal:

After reviewing Dr. Chi’s paper, "Recent Temporal Pattern Mining for Septic Shock Early Prediction", we found that their work cited an almost identical RTP F1-score of 0.868 using an observation window just 4 hours before diagnosis versus our F1-score of 0.860 using the same method 24 hours before diagnosis. That is a full 20 hours increase in advance notice with little drop-off in predictive ability. With such a minimal decrease in accuracy over such an extended period of time, we feel it would be reasonable to set a minimum acceptable F1-score and walk back the observation window an additional 24-48 hours in 4-hour intervals until the minimum F1-score is met. This will help to identify patients with sepsis that much earlier and before the disease has a chance to progress. 
